In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Import Libs**

In [2]:
import pandas as pd
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as IMBPipeline

**2. Load And Explore Data**

In [3]:
da1 = pd.read_csv(r"C:\Users\smdsu\Downloads\oil_spill (1).csv")

X = da1.drop('target', axis = 1)
y = da1['target']

**Split Data**

In [4]:
##X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

X.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)

**3. Define Model**

In [5]:
## We're going to use RandomForestClass as our model and SMOTE For Imbalanced Classific to oversample the minority cl
## Also we'll do feature scale using StandardScaler

pipeline = IMBPipeline(steps=[
    ('oversample', SMOTE(random_state=42)),       # Oversampling
    ('scaler', StandardScaler()),                 # Feature Scaling
    ('classifier', RandomForestClassifier(random_state=42))  # Classifier
])


**4. Importing Libraries and Evaluating using Cross-Validation Predition method**

In [6]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import average_precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
import numpy as np

print("Import successful.")

# StratifiedKFold to ensure each fold is representative of the overall class distribution
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
print("StratifiedKFold initialized.")

# Define the preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('poly', PolynomialFeatures(degree=2), slice(0, X.shape[1])), # Apply Polynomial Features to all columns
        ('pca', PCA(n_components=5), slice(0, X.shape[1]))            # Add PCA to reduce dimensionality
    ],
    remainder='passthrough'
)

# Define RandomForest model
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

# Define the pipeline with SMOTE, StandardScaler, and RandomForest
pipeline = ImbPipeline(steps=[
    ('smote', SMOTE(random_state=42)),  # Handle imbalanced data
    ('preprocessor', preprocessor),     # Advanced feature engineering with PCA and PolynomialFeatures
    ('scaler', StandardScaler()),       # Scaling features
    ('model', model)                    # Use RandomForestClassifier
])

# Cross-validated predictions
results = cross_validate(pipeline, X, y, cv=cv, scoring='average_precision', return_estimator=True)

# Aggregate predictions
y_pred_prob = np.zeros(len(y))
for est in results['estimator']:
    y_pred_prob += est.predict_proba(X)[:, 1]
y_pred_prob /= len(results['estimator'])

# Calculate Precision-Recall AUC
pr_auc = average_precision_score(y, y_pred_prob)
print(f'Precision-Recall AUC: {pr_auc:.4f}')


Import successful.
StratifiedKFold initialized.
Precision-Recall AUC: 0.9935


**5. Evaluating Accuracy**

In [7]:
from sklearn.metrics import precision_score, accuracy_score
import numpy as np

# Let's assume you have the true labels and predicted labels
y_true = np.array([1, 0, 1, 1, 0, 1, 0, 0, 1, 1])  # True labels
y_pred = np.array([1, 0, 1, 0, 0, 1, 0, 0, 1, 0])  # Predicted labels

# Calculate precision
precision = precision_score(y_true, y_pred)
print(f'Precision: {precision:.4f}')

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy:.4f}')


Precision: 1.0000
Accuracy: 0.8000
